# Scrapper de corpus de documents issus de la Bibliothèque numérique Heidelberg :


https://www.ub.uni-heidelberg.de/helios/Welcome.html

In [1]:
#@title Installation des bibliothèques nécessaires et connexion à un compte Google Drive dans lequel seront téléchargés les fichiers issues de la Bibliothèque.




''' 
Google Colab notebook.
Python == 3.7.11
4/1AX4XfWhU_KEt0Ic7mu0ihGu48zDU1HW-KwZWDTiihP2u-s4opuOCf87m8po
BaOIA - La Contemporaine - Université de Nanterre
'''

!pip install xmltodict==0.12.0
from google.colab import drive
from openpyxl import load_workbook
from bs4 import BeautifulSoup
import requests
from collections import OrderedDict
import xmltodict
from PIL import Image
import urllib
import zipfile
from collections import Counter
import webbrowser
import re
import bz2
import json
import glob
import time
import os
import urllib.request, urllib.error, urllib.parse
from lxml import etree
import xml.etree.ElementTree as ET
from urllib.request import urlopen

drive.mount('/content/drive/')
%cd /content/drive/My Drive/scrapper_heidelberg/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:



ValueError: ignored

In [ ]:
#@title Téléchargement du fichier de métadonnées du document ainsi que de la liste des volumes par années disponibles

#@markdown

#@markdown Le document nécessaire est un fichier au format XLSX (Excel) contenant la liste des liens vers les documents issus de la Bibliothèque numérique d'Heidelberg.

#@markdown

#@markdown Indiquer ci-dessous le chemin absolu vers le fichier excel (issu d'un GoogleDrive):



def mets_global(fichier_xls):
  """ Fonction qui prend en entrée un fichier excel contenant les liens vers les
  documents issus de la Bibliothèque Numérique d'Heidelberg. 
  Exemple de lien: https://digi.ub.uni-heidelberg.de/diglit/jugend
  Tous les liens sont indiqués tel quels das la première colonne du fichier,
  un par ligne.
  
  La fonction récupère le fichier METS-ALTO qui correponds à chaque document
  et le télécharge. Ce document contient toutes les métadonnées sur le document
  et permet d'avoir accès au nombres de volumes qui sont disponibles (ce sont les 
  liens visibles sur la page du document. 
  
  Le fichier téléchargé est un fichier au format XML.
  """

  global chemin_absolu_dossier
  global chemin_absolu_fichier_excel
  chemin_fichier_excel=os.path.abspath(fichier_excel)
  chemin_dossier = os.path.dirname(fichier_excel) 
  wb= load_workbook(fichier_xls)
  ws= wb['Feuil1']
  column_id = ws['A']
  id_list = [column_id[x].value for x in range(len(column_id))]
  for url_nom in id_list[0:1]:
    global nom_doc
    nom_doc = url_nom[41:]
    url_mets = "".join(url_nom + "/mets")
    global nom_fichier_mets
    nom_fichier_mets = "".join(nom_doc + "_METS.xml")
    %cd $chemin_dossier
    os.mkdir(nom_doc)
    %cd $nom_doc
    with open(nom_fichier_mets, "w") as fichier_xml:
      fichier_xml.write(urlopen(url_mets).read().decode('utf-8'))
      fichier_xml.close()
    global nom_dossier_mets_volumes
    nom_dossier_mets_volumes=os.path.dirname(nom_fichier_mets)
    chemin_dossier_nom_doc = os.path.abspath(nom_doc)
    

def mets_volumes(nom_fichier_mets):
  """ Fonction qui permet la récuprétaion des volumes disponibles et indiqués
  dans le fichier METS-ALTO du document.
  
  La fonction affiche la liste des volumes, ainsi que leurs dates pour permettre
  de choisir ceux que l'on souhaite télécharger ensuite. """

  global noms_volumes
  global liens_mets_volumes
  noms_volumes = []
  liens_mets_volumes = []
  with open(nom_fichier_mets, 'r+') as file_mets:
    my = xmltodict.parse(file_mets.read())
    my2 = dict(OrderedDict(my))
    for volume in my2['mets:mets']['mets:structMap']['mets:div']['mets:div']:
      volume = dict(OrderedDict(volume))
      for key, value in volume.items():
        if key == "@LABEL":
          noms_volumes.append(value)
        if key == "mets:mptr":
          value = dict(OrderedDict(value))
          for key2, value2 in value.items():
            if key2 == '@xlink:href':
              liens_mets_volumes.append(value2) 
  nombre_volumes = len(liens_mets_volumes)
  print("Le nombre de volumes de "+nom_doc+ " est de: "+str(nombre_volumes))
  print("Voici la liste des volumes disponibles : ")
  for ju, i in zip(noms_volumes, range(nombre_volumes)):
    print(i, ":", ju)

fichier_excel = "/content/drive/MyDrive/scrapper_heidelberg/test_heidelberg (2).xlsx" #@param {type:"string"}

mets_global(fichier_excel)
mets_volumes(nom_fichier_mets)

/content/drive/MyDrive/scrapper_heidelberg
/content/drive/MyDrive/scrapper_heidelberg/jugend
Le nombre de volumes de jugend est de: 73
Voici la liste des volumes disponibles : 
0 : Band 1; 1896, Band 1 (Nr. 1-26)
1 : Band 1; 1896, Band 2 (Nr. 27-52)
2 : Band 2; 1897, Band 1 (Nr. 1-26)
3 : Band 2; 1897, Band 2 (Nr. 27-52)
4 : Band 3; 1898, Band 1 (Nr. 1-26)
5 : Band 3; 1898, Band 2 (Nr. 27-52)
6 : Band 4; 1899, Band 1 (Nr. 1-26)
7 : Band 4; 1899, Band 2 (Nr. 27-52)
8 : Band 5; 1900, Band 1 (Nr. 1-26)
9 : Band 5; 1900, Band 2 (Nr. 27-52)
10 : Band 6; 1901, Band 1 (Nr. 1-26)
11 : Band 6; 1901, Band 2 (Nr. 27-52)
12 : Band 7; 1902, Band 1 (Nr. 1-26)
13 : Band 7; 1902, Band 2 (Nr. 27-52)
14 : Band 8; 1903, Band 1 (Nr. 1-26)
15 : Band 8; 1903, Band 2 (Nr. 27-52)
16 : Band 9; 1904, Band 1 (Nr. 1-26)
17 : Band 9; 1904, Band 2 (Nr. 27-52)
18 : Band 10; 1905, Band 1 (Nr. 1-26)
19 : Band 10; 1905, Band 2 (Nr. 27-52)
20 : Band 11; 1906, Band 1 (Nr. 1-26)
21 : Band 11; 1906, Band 2 (Nr. 27-52)
22 :

In [ ]:
#@title Téléchargement des fichiers XML-JPG-IIIF

#@markdown Indiquer le nombre du volume que l'on veut télécharger.

#@markdown Il est possible d'indiquer "tout" pour télécharger l'ensemble des volumes

#@markdown


## Récupération des fichiers BZ2 (alto-complets)

## Récupération des liens vers fichiers ALTO détaillés  pour chaque page de doc à partir des liens récuérés dans fichiers METS / volumes
def telecharger_fichiers_alto_bz2(nom_fichier_mets_volume):
  """Téléchargement des fichiers alto via la lien indiqué
  dans le document alto-xml ('nom_fichier_mets_volume') 
  regroupant toutes les pages d'un même volume. 
  Les fichier "xml-bz2" sont téléchargés sous forme
  compressée, la fonction décompresse les fichiers et supprime
  les fichiers compressés.
  Tous les fichiers / page sont sauvegardés dans le dossier
  alto-ml-bz2"""
  os.mkdir("alto-xml-bz2")
  os.chdir("alto-xml-bz2")
  global noms_altos
  global liens_altos
  noms_altos = []
  liens_altos = []
  for file_xml in glob.glob(nom_dossier_altos):
    with open(file_xml, 'r+') as file_alto:
      my = xmltodict.parse(file_alto.read())
      my2 = dict(OrderedDict(my))
      for key, value in my2['mets:mets']['mets:fileSec'].items():
          for ele in value:
            for key2, value2 in ele.items():
              if value2 == "FULLTEXT":
                paragraphe_ocr = ele
                for key3, value3 in paragraphe_ocr.items():
                  if type(value3) is list:
                    for element in value3:
                      for key4, value4 in element.items():
                        if key4 =="@ID":
                          noms_altos.append(value4)
                        if key4 == "mets:FLocat":
                          for key5, value5 in value4.items():
                            if key5 == '@xlink:href':
                              liens_altos.append(value5)
  for url_alto, nom_alto in zip(liens_altos, noms_altos):
    response = requests.get(url_alto).text
    lien = re.findall('\/data.*?\.[\w:]+\.xml.bz2', response)
    nom_fichier = "".join(nom_alto + "bz2")
    for lienn in lien:
      nouveau_lien = "".join("https://digi.ub.uni-heidelberg.de" + lienn[5:])
      urllib.request.urlretrieve(nouveau_lien, nom_fichier)
      global newfilepath
      newfilepath = os.path.join(nom_fichier + '.xml')
      with open(newfilepath, 'wb') as new_file, bz2.BZ2File(nom_fichier, 'rb') as file2:
        for data in iter(lambda : file2.read(100 * 1024), b''):
          new_file.write(data)
  global chemin_dossier_bz2
  chemin_dossier_bz2 = os.getcwd()
  chemin_dossier_pr_suppr = "".join(chemin_dossier_bz2 + '/*')
  for item in glob.glob(chemin_dossier_pr_suppr):
    if item.endswith(".xml"):
      pass
    else:
      os.remove(item)
  print("Téléchargement BZ2 OK")
  %cd ..\


def telecharger_fichiers_jpg(nom_fichier_mets_volume):
  """Téléchargement des document au format image (jpg) 
  via la lien indiqué dans le document alto-xml 
  ('nom_fichier_mets_volume') regroupant toutes les pages 
  d'un même volume.  """
  global noms_jpg
  global liens_jpg
  noms_jpg = []
  liens_jpg = []
  for file_xml in glob.glob(nom_dossier_altos):
    with open(file_xml, 'r+') as file_alto:
      my = xmltodict.parse(file_alto.read())
      my2 = dict(OrderedDict(my))
      for key, value in my2['mets:mets']['mets:fileSec'].items():
          for ele in value:
            for key2, value2 in ele.items():
              if value2 == "MIN":
                paragraphe_jpg = ele
                for key3, value3 in paragraphe_jpg.items():
                  if type(value3) is list:
                    for element in value3:
                      for key4, value4 in element.items():
                        if key4 =="@ID":
                          noms_jpg.append(value4)
                        if key4 == "mets:FLocat":
                          for key5, value5 in value4.items():
                            if key5 == '@xlink:href':
                              liens_jpg.append(value5)
  os.mkdir("jpg")
  os.chdir("jpg")
  for url_jpg, nom_jpg in zip(liens_jpg, noms_jpg):
    response2 = requests.get(url_jpg)
    global nom_fichier_jpg
    nom_fichier_jpg = "".join(nom_doc + "_"+ nom_volume + "_"+nom_jpg +".jpg")
    urllib.request.urlretrieve(url_jpg, nom_jpg)
  print("Téléchargement JPG OK")
  %cd ..\


def telecharger_IIIF(nom_fichier_mets_volume):
  """Téléchargement des manifestes IIIF 
  via la lien indiqué dans le document alto-xml 
  ('nom_fichier_mets_volume') regroupant toutes les pages 
  d'un même volume.  """
  global liens_iiif
  liens_iiif = []
  with open(nom_fichier_mets_volume, 'r') as gu:
    my =xmltodict.parse(gu.read())
    my2 = dict(OrderedDict(my))
    for key, value in my2['mets:mets'].items():
      if key == "mets:dmdSec":
        for ele in value:
          for key2, value2 in ele.items():
            if key2 == "mets:mdWrap":
              for key3, value3 in value2.items():
                if key3 =="mets:xmlData":
                  for key4, value4 in value3.items():
                    if key4 == "dc:hasFormat":
                      liens_iiif.append(value4)
  os.mkdir("iiif")
  os.chdir("iiif")
  global chemin_dossier_iiif
  chemin_dossier_iiif = os.getcwd()
  for lien in liens_iiif:
    response2 = requests.get(lien)
    global nom_fichier_iiif
    nom_fichier_iiif = "".join(nom_doc+"_manifest-IIIF"+".json")
    urllib.request.urlretrieve(lien, nom_fichier_iiif)
    global liens_iiif_pages
  liens_iiif_pages = []
  with open(nom_fichier_iiif, 'r') as hj:
    manifeste = json.load(hj)
  for key, value in manifeste.items():
    if key == "sequences":
      for ele in value:
        for key2, value2 in ele.items():
          if key2 == "canvases":
            for ele2 in value2:
              for key3, value3 in ele2.items():
                if key3 == "images":
                  for ele3 in value3:
                    for key4, value4 in ele3.items():
                      if key4 == "resource":
                        for key5, value5 in value4.items():
                          if key5 == "service":
                            for key6, value6 in value5.items():
                              if key6 == "@id":
                                liens_iiif_pages.append(value6)
  for lien, nom in zip(liens_iiif_pages, noms_altos):
    nouveau_lien = "".join(lien + "/info.json")
    global nom_fichier_unique_iiif
    nom_fichier_unique_iiif = "".join(nom+"_manifest-IIIF"+".json")
    response2 = requests.get(nouveau_lien)
    urllib.request.urlretrieve(nouveau_lien, nom_fichier_unique_iiif)
  print("Téléchargement IIIF OK")
  %cd ..\

def telecharger_illustrations(chemin_dossier_bz2, chemin_dossier_iiif):
  """Fonction de récupération des illustration contenues
  dans les documents. Les coordonnées sont récupérées dans 
  les fichiers xml-alto BZ2 et via les manifestes IIIF.
  
  /!\ La fonction nécessite de choisoir au préalable le
  téléchargement des fichiers alto-bz2 ainsi que celui des
  manifestes IIIF.
  
  """
  os.mkdir('illustrations')
  os.chdir('illustrations')
  chemin_dossier_illustrations = os.getcwd()
  dossier_alto = chemin_dossier_bz2+"/*.xml"
  dossier_iiif = chemin_dossier_iiif+"/*.json"
  for alto2, iiif in zip(sorted(glob.glob(dossier_alto)), sorted(glob.glob(dossier_iiif))):
    i = open(iiif, "r")
    iiif = json.load(i)
    for keys, values in iiif.items():
        if keys == "@id":
          base_lien_iiif = values
        if keys == "width":
          largeur_iiif = values  
    f=open(alto2, "r+")
    liste_images = []
    my = xmltodict.parse(f.read())
    alto = dict(OrderedDict(my))
    for key, value in alto.items():
      for key2, value2 in value.items():
        if key2 == "page":
          for key3, value3 in value2.items():
            if key3 == "@width":
              largeur_alto = value3
            if key3 == "block":
              if type(value3) == list:
                for ele in value3[-3:]:
                  for key4, value4 in ele.items():
                    if value4 == "Picture":
                      value3.remove(ele)
                for ele in value3:
                  for key4, value4 in ele.items():
                    if value4 == "Picture":
                      liste_images.append(value4)
                      for key4, value4 in ele.items():
                        if key4 == "@b":
                          b = value4
                        if key4 == "@l":
                          l = value4
                        if key4 == "@r":
                          r = value4
                        if key4 == "@t":
                          t = value4
                      horizontal = l
                      vertical = t
                      largeur= r
                      hauteur = b
                      largeur6 = int(largeur)-int(horizontal)
                      hauteur6 = int(hauteur)-int(vertical)
                      ratio = int(largeur_iiif) / int(largeur_alto)
                      horizontal2 = int(horizontal)*ratio
                      horizontal3=round(horizontal2)
                      vertical2 = int(vertical)*ratio
                      vertical3=round(vertical2)
                      largeur2 = int(largeur6)*ratio
                      largeur3 = round(largeur2)
                      hauteur2 = int(hauteur6)*ratio
                      hauteur3 = round(hauteur2)
                      num_illustration=(len(liste_images))
                      # reprends le nom du fichier alto ouvert:
                      nom_fichier_alto=os.path.basename(f.name)
                      nom_fichier_alto = nom_fichier_alto[:-4]
                      url = ''.join(base_lien_iiif+"/"+str(horizontal3)+","+str(vertical3)+","+str(largeur3)+","+str(hauteur3)+"/full/0/default.jpg")
                      nom_fichier = nom_fichier_alto+"-illustration-"+str(num_illustration)+".jpg"
                     # nom_doss = "illustrations"
                      cheminout = os.path.join(chemin_dossier_illustrations+"/"+nom_fichier)
                      urllib.request.urlretrieve(url, cheminout)

              else:
                for key4, value4 in value3.items():
                    if value4 == "Picture":
                      liste_images.append(value4)
                      for key4, value4 in ele.items():
                        if key4 == "@b":
                          b = value4
                        if key4 == "@l":
                          l = value4
                        if key4 == "@r":
                          r = value4
                        if key4 == "@t":
                          t = value4
                      horizontal = l
                      vertical = t
                      largeur= r
                      hauteur = b
                      largeur6 = int(largeur)-int(horizontal)
                      hauteur6 = int(hauteur)-int(vertical)
                      ratio = int(largeur_iiif) / int(largeur_alto)
                      horizontal2 = int(horizontal)*ratio
                      horizontal3=round(horizontal2)
                      vertical2 = int(vertical)*ratio
                      vertical3=round(vertical2)
                      largeur2 = int(largeur6)*ratio
                      largeur3 = round(largeur2)
                      hauteur2 = int(hauteur6)*ratio
                      hauteur3 = round(hauteur2)
                      num_illustration=(len(liste_images))
                      nom_fichier_alto=os.path.basename(f.name)
                      nom_fichier_alto = nom_fichier_alto[:-4]
                      url = ''.join(base_lien_iiif+"/"+str(horizontal3)+","+str(vertical3)+","+str(largeur3)+","+str(hauteur3)+"/full/0/default.jpg")
                      nom_fichier = nom_fichier_alto+"-illustration-"+str(num_illustration)+".jpg"
                      nom_doss = 'illustrations'
                      cheminout = os.path.join(nom_doss+"/"+nom_fichier)
                      urllib.request.urlretrieve(url, cheminout)
  print("Téléchargement Illustrations OK")
  %cd ..\

def extraire_texte_de_alto(nom_du_nouveau_fichier):

  ''' https://github.com/cneud/alto-ocr-text.git:
  
  Fonction qui permet d'extraire le texte OCRisé et contenu dans le fichier alto.
  Elle permet d'obtenir le text brut, sans aucune métadonnée ni indication de 
  mise en page. 
  Elle prend en entrée le chemin vers le/les fichier alto-xml.
  La sortie correponds à un fichier au format .txt du même nom que le fichier alto
  dont elle est extraite. '''

  nom_fichier = os.path.basename(nom_du_nouveau_fichier)
  nom_fichier_txt = nom_fichier[:-4]+".txt"

  namespace = {'alto-1': 'http://schema.ccs-gmbh.com/ALTO',
             'alto-2': 'http://www.loc.gov/standards/alto/ns-v2#',
             'alto-3': 'http://www.loc.gov/standards/alto/ns-v3#'}
  tree = ET.parse(nom_du_nouveau_fichier)
  xmlns = tree.getroot().tag.split('}')[0].strip('{')
  if xmlns in namespace.values():
    for lines in tree.iterfind('.//{%s}TextLine' % xmlns):
        with open(nom_fichier_txt, 'a') as op:
          for line in lines.findall('{%s}String' % xmlns):
            text = line.attrib.get('CONTENT') + ' '
            op.write(text)
  print("Téléchargement alto-txt OK")


def telecharger_fichiers_altos_ocr(nom_fichier_mets_volume):

  """ Fonction qui permet de récupérer les liens vers les fichiers xml-alto (OCR) par pages. 
  Ces fichiers ne contiennent pas d'autres indications que le texte OCRisé et leur placement,
  contrairement aux fichiers xml-alto ("bz2") qui contiennent aussi des indications sur les illustrations
  présentes sur les pages 
  Les liens sont récupérés sur le fichier mets-alto qui correspond au volume du document.
  Les fichiers xml-alto sont téléchargés automatiquement grâce à ces liens. 
  La fonction prends en entrée le chemin vers le fichier mets du volume. """

  f=open(nom_fichier_mets_volume, "r")
  chemin_dossier1 = os.getcwd()
  os.mkdir("txt-ocr")
  os.chdir("txt-ocr")
  chemin_dossier_altos = os.getcwd()
  my = xmltodict.parse(f.read())
  alto = dict(OrderedDict(my))
  liste_liens_altos_petits =[]
  liste_noms_altos_petits = []
  for key, value in alto.items():
    for key2, value2 in value.items():
      if key2 == "mets:fileSec":
        for key3, value3 in value2.items():
          if key3 == "mets:fileGrp":
            for ele in value3:
              for key4, value4 in ele.items():
                if value4 == "FULLTEXT":
                  for key5, value5 in ele.items():
                    if key5 == "mets:file":
                      for ele2 in value5:
                        for key6, value6 in ele2.items():
                          if key6 == '@ID':
                            liste_noms_altos_petits.append(value6)
                          if key6 == "mets:FLocat":
                            for key7, value7 in value6.items():
                              if key7 == "@xlink:href":
                                liste_liens_altos_petits.append(value7)
  for liennn, nommm in zip(liste_liens_altos_petits, liste_noms_altos_petits):
    global nom_du_nouveau_fichier
    nom_du_nouveau_fichier = nommm + ".xml"
    response2 = requests.get(liennn)
    urllib.request.urlretrieve(liennn, nom_du_nouveau_fichier)
    extraire_texte_de_alto(nom_du_nouveau_fichier)
    chemin_dossier_ = os.getcwd()
    chemin_dossier_pr_suppr = "".join(chemin_dossier_ + '/*')
    for item in glob.glob(chemin_dossier_pr_suppr):
      if item.endswith(".txt"):
        pass
      else:
        os.remove(item)
  print("Téléchargement txt OK")
  %cd ..\
  



def get_mets_volumes(liens_mets_volumes, noms_volumes):
  print("Liste avec tous les volumes" + str(noms_volumes))
  global nom_volume
  global nom_fichier_mets_volume
  global nom_dossier_altos
  volume_a_telecharger = "tout" #@param {type:"string"}
  #digitals = re.findall(r'^[0-9]*$', volume_a_telecharger)
  telecharger_les_fichiers_alto = True #@param {type:"boolean"}
  telecharger_les_illustrations = True #@param {type:"boolean"}
  telecharger_les_fichiers_images = True #@param {type:"boolean"}
  telecharger_les_fichiers_iiif = True #@param {type:"boolean"}
  telecharger_les_fichiers_texte = True #@param {type:"boolean"}
  try:
    digital = int(volume_a_telecharger)
    url_mets_volumes = liens_mets_volumes[int(volume_a_telecharger)]
    nom_volume = noms_volumes[int(volume_a_telecharger)]
    nom_volume = re.sub(r'/', '-', nom_volume)
    os.mkdir(nom_volume)
    os.chdir(nom_volume)
    response2 = requests.get(url_mets_volumes)
    global nom_fichier_mets_volume
    nom_fichier_mets_volume = "".join(nom_doc + "_"+ nom_volume +"_METS.xml")
    with open(nom_fichier_mets_volume, "w") as fichier_xml:
      fichier_xml.write(urlopen(url_mets_volumes).read().decode('utf-8'))
      fichier_xml.close()
      nom_dossier_altos=os.path.abspath(nom_fichier_mets_volume)
      if telecharger_les_fichiers_alto == True:
        telecharger_fichiers_alto_bz2(nom_fichier_mets_volume) 
      if telecharger_les_fichiers_iiif == True:
        telecharger_IIIF(nom_fichier_mets_volume)
      if telecharger_les_fichiers_images == True:
        telecharger_fichiers_jpg(nom_fichier_mets_volume)
      if telecharger_les_illustrations == True:
        telecharger_illustrations(chemin_dossier_bz2, chemin_dossier_iiif)
      if telecharger_les_fichiers_texte == True:
        telecharger_fichiers_altos_ocr(nom_fichier_mets_volume)
    
  except ValueError as pu:
    for url_mets_volumes, nom_volume in zip(liens_mets_volumes, noms_volumes):
      nom_volume = re.sub(r'/', '-', nom_volume)
      os.mkdir(nom_volume)
      os.chdir(nom_volume)
      response2 = requests.get(url_mets_volumes)
      nom_fichier_mets_volume = "".join(nom_doc + "_"+ nom_volume +"_METS.xml")
      with open(nom_fichier_mets_volume, "w") as fichier_xml:
        fichier_xml.write(urlopen(url_mets_volumes).read().decode('utf-8'))
        fichier_xml.close()
      nom_dossier_altos=os.path.abspath(nom_fichier_mets_volume)
      if telecharger_les_fichiers_alto == True:
        telecharger_fichiers_alto_bz2(nom_fichier_mets_volume) 
      if telecharger_les_fichiers_iiif == True:
        telecharger_IIIF(nom_fichier_mets_volume)
      if telecharger_les_fichiers_images == True:
        telecharger_fichiers_jpg(nom_fichier_mets_volume)
      if telecharger_les_illustrations == True:
        telecharger_illustrations(chemin_dossier_bz2, chemin_dossier_iiif)
      if telecharger_les_fichiers_texte == True:
        telecharger_fichiers_altos_ocr(nom_fichier_mets_volume)


get_mets_volumes(liens_mets_volumes, noms_volumes)



Liste avec tous les volumes['Band 1; 1896, Band 1 (Nr. 1-26)', 'Band 1; 1896, Band 2 (Nr. 27-52)', 'Band 2; 1897, Band 1 (Nr. 1-26)', 'Band 2; 1897, Band 2 (Nr. 27-52)', 'Band 3; 1898, Band 1 (Nr. 1-26)', 'Band 3; 1898, Band 2 (Nr. 27-52)', 'Band 4; 1899, Band 1 (Nr. 1-26)', 'Band 4; 1899, Band 2 (Nr. 27-52)', 'Band 5; 1900, Band 1 (Nr. 1-26)', 'Band 5; 1900, Band 2 (Nr. 27-52)', 'Band 6; 1901, Band 1 (Nr. 1-26)', 'Band 6; 1901, Band 2 (Nr. 27-52)', 'Band 7; 1902, Band 1 (Nr. 1-26)', 'Band 7; 1902, Band 2 (Nr. 27-52)', 'Band 8; 1903, Band 1 (Nr. 1-26)', 'Band 8; 1903, Band 2 (Nr. 27-52)', 'Band 9; 1904, Band 1 (Nr. 1-26)', 'Band 9; 1904, Band 2 (Nr. 27-52)', 'Band 10; 1905, Band 1 (Nr. 1-26)', 'Band 10; 1905, Band 2 (Nr. 27-52)', 'Band 11; 1906, Band 1 (Nr. 1-26)', 'Band 11; 1906, Band 2 (Nr. 27-52)', 'Band 12; 1907, Band 1 (Nr. 1-27)', 'Band 12; 1907, Band 2 (Nr. 26-52)', 'Band 13; 1908, Band 1 (Nr. 1-27)', 'Band 13; 1908, Band 2 (Nr. 27-52)', 'Band 14; 1909, Band 1 (Nr. 1-26)', 'Band

FileNotFoundError: ignored

In [ ]:
###
## dans fichier bz2 xml illustratios en ltrb / horizontal vertical largeur hauteur
#horizontal = l
#vertical = t
#largeur = r-horizontal
#hauteur = b-vertical
#vérifier ratio + intégrer
%cd /content/drive/MyDrive/scrapper_heidelberg/test_images/test_3/
#iiif = '/content/drive/MyDrive/scrapper_heidelberg/test_images/test_4/fileocr00012_manifest-IIIF.json'
#alto2 = '/content/drive/MyDrive/scrapper_heidelberg/test_images/test_4/fileocr00012bz2.xml'

def telecharger_illustrations(chemin_dossier, nom_dossier_iiif):
  dossier_alto = chemin_dossier+"/*.xml"
  dossier_iiif = chemin_iiif+"/*.json"
  for alto2, iiif in zip(sorted(glob.glob(dossier_alto)), sorted(glob.glob(dossier_iiif))):
    i = open(iiif, "r")
    iiif = json.load(i)
    for keys, values in iiif.items():
        if keys == "@id":
          base_lien_iiif = values
        if keys == "width":
          largeur_iiif = values  
    f=open(alto2, "r+")
    liste_images = []
    my = xmltodict.parse(f.read())
    alto = dict(OrderedDict(my))
    for key, value in alto.items():
      for key2, value2 in value.items():
        if key2 == "page":
          for key3, value3 in value2.items():
            if key3 == "@width":
              largeur_alto = value3
            if key3 == "block":
              if type(value3) == list:
                for ele in value3[-3:]:
                  for key4, value4 in ele.items():
                    if value4 == "Picture":
                      value3.remove(ele)
                for ele in value3:
                  for key4, value4 in ele.items():
                    if value4 == "Picture":
                      liste_images.append(value4)
                      for key4, value4 in ele.items():
                        if key4 == "@b":
                          b = value4
                        if key4 == "@l":
                          l = value4
                        if key4 == "@r":
                          r = value4
                        if key4 == "@t":
                          t = value4
                      horizontal = l
                      vertical = t
                      largeur= r
                      hauteur = b
                      largeur6 = int(largeur)-int(horizontal)
                      hauteur6 = int(hauteur)-int(vertical)
                      ratio = int(largeur_iiif) / int(largeur_alto)
                      horizontal2 = int(horizontal)*ratio
                      horizontal3=round(horizontal2)
                      vertical2 = int(vertical)*ratio
                      vertical3=round(vertical2)
                      largeur2 = int(largeur6)*ratio
                      largeur3 = round(largeur2)
                      hauteur2 = int(hauteur6)*ratio
                      hauteur3 = round(hauteur2)
                      num_illustration=(len(liste_images))
                      nom_fichier_alto=os.path.basename(f.name)
                      nom_fichier_alto = nom_fichier_alto[:-4]
                      url = ''.join(base_lien_iiif+"/"+str(horizontal3)+","+str(vertical3)+","+str(largeur3)+","+str(hauteur3)+"/full/0/default.jpg")
                      nom_fichier = nom_fichier_alto+"-illustration-"+str(num_illustration)+".jpg"
                      nom_doss = "illustrations3"
                      cheminout = os.path.join(nom_doss+"/"+nom_fichier)
                      urllib.request.urlretrieve(url, cheminout)

              else:
                for key4, value4 in value3.items():
                    if value4 == "Picture":
                      liste_images.append(value4)
                      for key4, value4 in ele.items():
                        if key4 == "@b":
                          b = value4
                        if key4 == "@l":
                          l = value4
                        if key4 == "@r":
                          r = value4
                        if key4 == "@t":
                          t = value4
                      horizontal = l
                      vertical = t
                      largeur= r
                      hauteur = b
                      largeur6 = int(largeur)-int(horizontal)
                      hauteur6 = int(hauteur)-int(vertical)
                      ratio = int(largeur_iiif) / int(largeur_alto)
                      horizontal2 = int(horizontal)*ratio
                      horizontal3=round(horizontal2)
                      vertical2 = int(vertical)*ratio
                      vertical3=round(vertical2)
                      largeur2 = int(largeur6)*ratio
                      largeur3 = round(largeur2)
                      hauteur2 = int(hauteur6)*ratio
                      hauteur3 = round(hauteur2)
                      num_illustration=(len(liste_images))
                      nom_fichier_alto=os.path.basename(f.name)
                      nom_fichier_alto = nom_fichier_alto[:-4]
                      url = ''.join(base_lien_iiif+"/"+str(horizontal3)+","+str(vertical3)+","+str(largeur3)+","+str(hauteur3)+"/full/0/default.jpg")
                      nom_fichier = nom_fichier_alto+"-illustration-"+str(num_illustration)+".jpg"
                      nom_doss = 'illustrations'
                      cheminout = os.path.join(nom_doss+"/"+nom_fichier)
                      urllib.request.urlretrieve(url, cheminout)

    





/content/drive/MyDrive/scrapper_heidelberg/test_images/test_3


KeyboardInterrupt: ignored